# Structure Learning

## Set sys vars

In [1]:
import os,sys
from pathlib import Path
currentdir = Path(os.getcwd())
sys.path.insert(0, os.path.dirname(currentdir.parent))
sys.path.insert(0, os.path.dirname(currentdir))
print(str(sys.path[0:3]))

['C:\\Users\\Roman\\Desktop\\GitBamt\\BAMT', 'C:\\Users\\Roman\\Desktop\\GitBamt', 'C:\\Users\\Roman\\Desktop\\GitBamt\\BAMT\\tutorials']


## Import modules

In [2]:
%%time
%matplotlib inline
import BAMT.Networks as Nets
import BAMT.Preprocessors as pp

import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

Wall time: 6.77 s


## Preprocessing

In [3]:
hack = pd.read_csv(r'../Data/hack_processed_with_rf.csv')

In [4]:
cols = ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Gross','Netpay','Porosity','Permeability', 'Depth']
hack = hack[cols]

In [5]:
encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

p = pp.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])
discretized_data, est = p.apply(hack)

## Initializing Bayessian Network

<p> There are 3 type of Bayessian Networks - DiscreteBN, ContinuousBN, HybridBN. <br>
Note that if you pass discrete data to Continoust BN, you get an error.<br><br>
For ContinousBN user can choose whether use mixture nodes or not, for Hybrid user can restrict/allow using logit/mixture nodes.<p> 

List of scoring_functions BAMT can deal with: <br>
1. Group 1
    - Mutual Information (MI)
    - LL
    - BIC
    - AIC
2. K2Score <br><br>
<p> For group 1 user can pass tuple as ('MI',). For K2Score user must import K2Scorer and pass ("K2", K2Scorer) <p>

In [6]:
bn = Nets.HybridBN(has_logit=True, use_mixture=True) # init BN
info = p.info # mapping of nodes (Dict["types": Dict[node_name: type], "signs": Dict[node_name: sign]])

Structural learning contains from two parts: building nodes and building edges. <br><br>

First stage: <br>
Instance of bayessian network initialized a primary nodes inside with two types ('Discrete' and 'Gaussian') according descriptor's info<br>
Second stage: <br>
Instance of bayessian network rewrites nodes according their parents and conditions (parameters)

In [7]:
bn.add_nodes(info)
# DATA PREPROCESSED MUST BE THERE
bn.add_edges(discretized_data, scoring_function=('MI',)) # use mutual information sf implemented in BAMT

In [8]:
bn.get_info()

,name,node_type,data_type,parents,parents_types
0,Tectonic regime,Discrete,disc,[],[]
1,Period,ConditionalLogit,disc,"[Depth, Structural setting, Lithology]","[cont, disc, disc]"
2,Lithology,ConditionalLogit,disc,"[Netpay, Structural setting]","[cont, disc]"
3,Structural setting,Logit,disc,[Permeability],[cont]
4,Gross,MixtureGaussian,cont,[Porosity],[cont]
5,Netpay,MixtureGaussian,cont,[Permeability],[cont]
6,Porosity,ConditionalMixtureGaussian,cont,[Tectonic regime],[disc]
7,Permeability,MixtureGaussian,cont,[Gross],[cont]
8,Depth,MixtureGaussian,cont,[Gross],[cont]


In [9]:
bn.plot('Test1')

2022-01-12 16:12:49,177 | ERROR    | Networks.py-plot-0207 | This version allows only html format.


In [10]:
# Now let's plot graph
bn.plot('Simple.html')

## Parameters in structure learning

<p> There are 5 parameters user can tune: init_nodes, init_edges, white_list, remove_init_edges, bl_add. <p>

### Init_nodes

This one defines root nodes (thus they haven't parents at all).

In [11]:
print(bn.nodes)

[Tectonic regime, Period, Lithology, Structural setting, Gross, Netpay, Porosity, Permeability, Depth]


In [12]:
params = {'init_nodes': ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Gross']}
bn.add_edges(discretized_data, scoring_function=('MI',), params=params)
bn.plot('init_nodes.html')

### Init_edges

<p> This one defines edges from which learning procedure starts <p>

In [13]:
params = {'init_nodes': ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Gross'],
          'init_edges':[('Period', 'Permeability'), ('Structural setting', 'Netpay'), ('Gross', 'Permeability')],}
bn.add_edges(discretized_data, scoring_function=('MI',), params=params)
bn.plot('init_edges.html')

### white_list

Strictly set edges where algoritm must learn

In [14]:
params = {'init_nodes': ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Gross'],
         'white_list': [('Period', 'Permeability'), ('Structural setting', 'Netpay'), ('Gross', 'Permeability')]}
bn.add_edges(discretized_data, scoring_function=('MI',), params=params)
bn.plot('whihe_list.html')

### bl_add

Restrict edges

In [16]:
params = {'init_nodes': ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Gross'],
         'white_list': [('Period', 'Permeability'), ('Structural setting', 'Netpay'), ('Gross', 'Permeability')],
         'bl_add':[('Structural setting', 'Netpay')]}
bn.add_edges(discretized_data, scoring_function=('MI',), params=params)
bn.plot('bl_add.html')

### Remove_init_edges

Allow algorithm to remove edges defined by user.

In [17]:
params = {'init_nodes': ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Gross'],
          'init_edges':[('Period', 'Permeability'), ('Structural setting', 'Netpay'), ('Gross', 'Permeability')],
         'remove_init_edges':True}
bn.add_edges(discretized_data, scoring_function=('MI',), params=params)
bn.plot('remove_init.html')